# Dataset exploration

In [0]:
%tensorflow_version 2.x

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

TensorFlow 2.x selected.


In [0]:
!git clone https://github.com/EmptyJackson/datasci-practical-b
%cd datasci-practical-b/

Cloning into 'datasci-practical-b'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 54 (delta 21), reused 39 (delta 12), pack-reused 0
Unpacking objects: 100% (54/54), done.
/content/datasci-practical-b


In [0]:
!git pull

Already up to date.


In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/ekochmar/cl-datasci-pnp/master/Final_assignment/diabetes/diabetic_data_original.csv")
#data = pd.read_csv("https://raw.githubusercontent.com/ekochmar/cl-datasci-pnp/master/Final_assignment/diabetes/diabetic_data_balanced.csv")
ids = pd.read_csv("https://raw.githubusercontent.com/ekochmar/cl-datasci-pnp/master/Final_assignment/diabetes/IDs_mapping.csv")

In [0]:
data[:11]

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,?,?,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,?,?,70,1,21,0,0,0,414,411,V45,7,None,None,Steady,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,?,?,73,0,12,0,0,0,428,492,250,8,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,?,?,68,2,28,0,0,0,398,427,38,8,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,?,InternalMedicine,33,3,18,0,0,0,434,198,486,8,None,None,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [0]:
data['time_in_hospital'].isnull().values.any()

False

In [0]:
from transformers import pipeline
df = pipeline.fit_transform(data)
df

,encounter_id,patient_nbr,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,glyburide-metformin,change,diabetesMed,readmit_bin,gender_bin,age_mid,number_emergency_lg,number_inpatient_lg,number_outpatient_lg,admission_type_id_1,admission_type_id_2,admission_type_id_3,admission_source_id_1,admission_source_id_2,admission_source_id_3,admission_source_id_4,admission_source_id_5,admission_source_id_6,admission_source_id_7,discharge_disposition_id_1,discharge_disposition_id_2,discharge_disposition_id_3,...,Asian,Caucasian,InternalMedicine,Emergency/Trauma,Family/GeneralPractice,Cardiology,Surgery-General,Nephrology,Orthopedics,Orthopedics-Reconstructive,Radiologist,Pulmonology,Psychiatry,Urology,ObstetricsandGynecology,Surgery-Cardiovascular/Thoracic,Gastroenterology,Surgery-Vascular,Surgery-Neuro,PhysicalMedicineandRehabilitation,Oncology,Pediatrics,Hematology/Oncology,Neurology,Pediatrics-Endocrinology,Otolaryngology,Endocrinology,Surgery-Thoracic,Psychology,Podiatry,Surgery-Cardiovascular,Pediatrics-CriticalCare,Hematology,Gynecology,Hospitalist,Radiology,max_glu_serum_>200,max_glu_serum_>300,A1Cresult_>7,A1Cresult_>8
12,40926,85504905,7,60,0,15,8,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,45,0.693147,0.000000,0.000000,1,0,0,0,0,0,0,0,0,1,0,0,1,...,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
27,248916,115196778,2,25,2,11,3,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,55,0.000000,0.000000,0.000000,1,0,0,1,0,0,0,0,0,0,1,0,0,...,0.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
28,250872,41606064,10,53,0,20,6,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,25,0.000000,0.000000,0.000000,0,1,0,0,1,0,0,0,0,0,1,0,0,...,0.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0
32,260166,80845353,6,27,0,16,8,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,75,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0,0,1,1,0,0,...,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
33,293058,114715242,5,37,0,18,8,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,65,0.000000,0.000000,0.000000,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101757,443847176,50375628,6,45,1,25,9,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,65,0.693147,1.098612,1.386294,1,0,0,0,0,0,0,0,0,1,1,0,0,...,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0
101758,443847548,100162476,3,51,0,16,9,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,75,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0,0,1,0,0,1,...,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0
101759,443847782,74694222,5,33,3,18,9,0,0,0,0,0,0,0,0,0,0,1,0,1,1,2,0,85,0.000000,0.693147,0.000000,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0
101760,443854148,41088789,1,53,0,9,13,1,0,0,0,0,0,0,0,0,0,1,0,0,1,2,1,75,0.000000,0.000000,0.693147,1,0,0,0,0,0,0,0,0,1,1,0,0,...,0.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0


In [0]:
sum(df['patient_nbr'].value_counts() >= 5)

1590

## Selecting multi-task helper attribute

### Same visit correlation

In [0]:
corrs = pat_ext.corr()['time_in_hospital']
corrs.map(lambda x : x).abs().sort_values(ascending = False)[:10]

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_df)
pd.DataFrame(x_scaled, columns=x_df.columns)

### Inter-visit correlation

In [0]:
from datasets import get_sequence_dataset, get_oneshot_dataset
patient, outcome = get_oneshot_dataset(df)

patient = pd.DataFrame(patient, columns=df.columns[2:])
outcome = pd.Series(outcome, name='next_time_in_hospital')

In [0]:
pat_ext = patient.copy()
pat_ext['next_time_in_hospital'] = outcome

In [0]:
corrs = pat_ext.corr()['next_time_in_hospital']
corrs.map(lambda x : x).abs().sort_values(ascending = False)[:10]

## Sequence lengths for RNN

In [0]:
from datasets import get_sequence_dataset

x, y = get_sequence_dataset(df)
seq_len_count = {}
for seq in x:
    if not len(seq) in seq_len_count:
        seq_len_count[len(seq)] = 1
    else:
        seq_len_count[len(seq)] = seq_len_count[len(seq)] + 1

In [0]:
seq_len_count = {k: v for k, v in sorted(seq_len_count.items(), key=lambda item: item[1], reverse=True)}
seq_len_count

In [0]:
max_len = 5
trunc_counts = {}
for seq_len, count in seq_len_count.items():
    if seq_len < max_len:
        trunc_counts[seq_len] = count
    else:
        if not max_len in trunc_counts:
            trunc_counts[max_len] = 0
        trunc_counts[max_len] = trunc_counts[max_len] + count

fig = plt.figure(figsize=(7,5))
ax = plt.subplot(1,1,1)
ax.bar([i+1 for i in range(len(trunc_counts))], list(trunc_counts.values()))
ax.set_ylabel('Patient records')
ax.set_xlabel('Number of readmissions')
ax.set_xticklabels([0,1,2,3,4,'>4'])
plt.show()
#plt.savefig('num_readmissions.png', dpi=150)